In [1]:
import csv
import heapq

## REQUIRED HELPER FUNCTIONS

In [2]:
class Node:
    def __init__(self, name, g_cost, h_cost, parent=None):
        self.name = name
        self.g_cost = g_cost  # Cost from start to current node
        self.h_cost = h_cost  # Heuristic cost from current node to goal
        self.f_cost = g_cost + h_cost  # Total cost
        self.parent = parent

    def __lt__(self, other):
        return self.f_cost < other.f_cost

In [3]:
def load_nodes(filename):
    nodes = {}
    with open(filename, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            nodes[row['node_name']] = {
                'type': row['type'],
                'x': float(row['x']),
                'y': float(row['y'])
            }
    return nodes

In [4]:
def load_edges(filename):
    edges = {}
    with open(filename, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if row['from_node'] not in edges:
                edges[row['from_node']] = []
            edges[row['from_node']].append((row['to_node'], int(row['cost']), row['mode'], int(row['time'])))
    return edges

In [5]:
def load_transfers(filename):
    transfers = {}
    with open(filename, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if row['node_name'] not in transfers:
                transfers[row['node_name']] = []
            transfers[row['node_name']].append({
                'from_mode': row['from_mode'],
                'to_mode': row['to_mode'],
                'cost': int(row['transfer_cost']),
                'time': float(row['transfer_time'])
            })
    return transfers

In [6]:
def load_dynamic_factors(filename):
    factors = {}
    with open(filename, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            factors[(row['from_node'], row['to_node'])] = {
                'accident_risk': float(row['accident_risk']),
                'congestion_factor': float(row['congestion_factor']),
                'weather_delay': float(row['weather_delay']),
            }
    return factors

In [7]:
def adjust_costs_dynamic(node, to_node, base_cost, current_conditions):
    """
    Adjust the cost dynamically based on conditions like accidents, weather, etc.
    """
    factor = current_conditions.get((node, to_node), {})
    accident_risk = factor.get('accident_risk', 1)
    congestion_factor = factor.get('congestion_factor', 1)
    weather_delay = factor.get('weather_delay', 1)

    w1=0.4 #accident risk
    w2=0.5 #congestion factor
    w3=0.6 #weather delay
    
    adjusted_cost = base_cost * (1 + w1 * accident_risk + w2 * congestion_factor + w3 * weather_delay)

    return adjusted_cost

In [8]:
def manhattan_distance(node, goal_node):
    # Simplified Manhattan distance as heuristic
    return abs(node['x'] - goal_node['x']) + abs(node['y'] - goal_node['y'])


## PLOTTING THE GRAPH

In [14]:
#!pip install networkx

In [15]:
#!pip install --user matplotlib

In [9]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
# Function to convert latitude and longitude to Mercator projection
def mercator_projection(lat, lon):
    R = 6378137  # Radius of the Earth in meters
    x = R * np.radians(lon)
    y = R * np.log(np.tan(np.pi / 4 + np.radians(lat) / 2))
    x=x*50
    y=y*50
    return x, y

# Read the nodes data (replace with your CSV path)
nodes = pd.read_csv('nodes.csv')

# Apply Mercator projection to convert lat/lon to x/y coordinates
nodes['scaled_x'], nodes['scaled_y'] = zip(*nodes.apply(lambda row: mercator_projection(row['x'], row['y']), axis=1))

# Save to a new CSV file, retaining the original data and adding projected coordinates
nodes.to_csv('mercator_nodes.csv', index=False)

# Read the edges data (replace with your CSV path)
edges = pd.read_csv('edges.csv')

# Create a graph using networkx
G = nx.Graph()

# Add nodes with positions (Mercator projected coordinates)
for index, row in nodes.iterrows():
    G.add_node(row['node_name'], pos=(row['scaled_x'], row['scaled_y']))

# Add edges with time and cost (from the original CSV)
for index, row in edges.iterrows():
    if row['from_node'] in G.nodes and row['to_node'] in G.nodes:
        G.add_edge(row['from_node'], row['to_node'], weight=row['cost'], time=row['time'])
    else:
        print(f"Warning: Edge from {row['from_node']} to {row['to_node']} references a node not in the graph")

# Get positions for drawing the graph
pos = nx.get_node_attributes(G, 'pos')

# Draw the graph
plt.figure(figsize=(20, 20))
nx.draw(G, pos, with_labels=True, node_size=500, node_color='skyblue', font_size=10, font_weight='bold')

# Draw edge labels with costs
edge_labels = {(row['from_node'], row['to_node']): row['cost'] for _, row in edges.iterrows() if row['from_node'] in G.nodes and row['to_node'] in G.nodes}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

plt.title("Graph of Nodes with Mercator Projection Coordinates")
plt.show()

## TAKING THE INPUT

In [18]:
#%pip install ipywidgets 

In [39]:
from IPython.display import display
import ipywidgets as widgets

# Load unique city names from file
with open('unique_city_names.txt', 'r') as file:
    unique_cities = [line.strip() for line in file]

# Create dropdown widgets for start and goal cities
start_dropdown = widgets.Dropdown(
    options=unique_cities,
    description='Start:',
    value='Guwahati'
)

goal_dropdown = widgets.Dropdown(
    options=unique_cities,
    description='Goal:',
    value='Guwahati'
)

# Create an integer slider for the time window
time_window_slider = widgets.IntSlider(
    value=100,
    min=1,
    max=1000,
    step=10,
    description='Time Window:'
)

# Display the widgets
display(start_dropdown, goal_dropdown, time_window_slider)

# Global variables to store the selected values
selected_start = start_dropdown.value
selected_goal = goal_dropdown.value
selected_time_window = time_window_slider.value

# Function to update the global variables when the widget value changes
def update_start(change):
    global selected_start
    selected_start = change['new']

def update_goal(change):
    global selected_goal
    selected_goal = change['new']

def update_time_window(change):
    global selected_time_window
    selected_time_window = change['new']

# Add observers to the widgets to update global variables dynamically
start_dropdown.observe(update_start, names='value')
goal_dropdown.observe(update_goal, names='value')
time_window_slider.observe(update_time_window, names='value')

# Now the values of selected_start, selected_goal, and selected_time_window
# will be globally accessible and updated dynamically when the widgets are changed.


Dropdown(description='Start:', options=('Guwahati', 'Dispur', 'Tezpur', 'Jorhat', 'Dibrugarh', 'Silchar', 'Tin…

Dropdown(description='Goal:', options=('Guwahati', 'Dispur', 'Tezpur', 'Jorhat', 'Dibrugarh', 'Silchar', 'Tins…

IntSlider(value=100, description='Time Window:', max=1000, min=1, step=10)

## A* ALGORITHM

In [14]:
def a_star_search_dynamic(start, goal, nodes, edges, transfers, time_window, current_conditions):
    open_list = []
    closed_list = set()
    
    start_node = Node(start, 0, manhattan_distance(nodes[start], nodes[goal]))
    heapq.heappush(open_list, start_node)
    
    while open_list:
        current_node = heapq.heappop(open_list)
        
        if current_node.name == goal:
            path = []
            while current_node:
                path.append(current_node.name)
                current_node = current_node.parent
            return path[::-1]
        
        closed_list.add(current_node.name)
        
        if current_node.name not in edges:
            continue
        
        for neighbor, base_cost, mode, travel_time in edges[current_node.name]:
            if neighbor in closed_list:
                continue
            
            adjusted_cost = adjust_costs_dynamic(current_node.name, neighbor, base_cost, current_conditions)
            g_cost = current_node.g_cost + adjusted_cost
            
            if g_cost > time_window:
                continue
            
            h_cost = manhattan_distance(nodes[neighbor], nodes[goal])
            neighbor_node = Node(neighbor, g_cost, h_cost, current_node)
            heapq.heappush(open_list, neighbor_node)
    
    return None

In [15]:
def a_star_search_dynamic_print_path(start, goal, nodes, edges, transfers, time_window, current_conditions):
    open_list = []
    closed_list = set()
    
    try:
        start_node = Node(start, 0, manhattan_distance(nodes[start], nodes[goal]))
    except KeyError as e:
        print(f"Error: {e} not found in nodes dictionary")
        return None
    
    heapq.heappush(open_list, start_node)
    
    while open_list:
        current_node = heapq.heappop(open_list)
        
        if current_node.name == goal:
            path = []
            total_cost = current_node.g_cost
            while current_node:
                path.append(current_node.name)
                current_node = current_node.parent
            print(f"Path taken: {path[::-1]}")
            print(f"Total path cost: {total_cost}")
            return path[::-1]
        
        closed_list.add(current_node.name)
        
        if current_node.name not in edges:
            continue
        
        for neighbor, base_cost, mode, travel_time in edges[current_node.name]:
            if neighbor in closed_list:
                continue
            
            adjusted_cost = adjust_costs_dynamic(current_node.name, neighbor, base_cost, current_conditions)
            g_cost = current_node.g_cost + adjusted_cost
            
            if g_cost > time_window:
                continue
            
            h_cost = manhattan_distance(nodes[neighbor], nodes[goal])
            neighbor_node = Node(neighbor, g_cost, h_cost, current_node)
            heapq.heappush(open_list, neighbor_node)
            
            print(f"Considering path from {current_node.name} to {neighbor} with cost {adjusted_cost} (mode: {mode})")
    
    print("No path found")
    return None

## RUN THE CODE

In [41]:
# Load data from CSV files
nodes = load_nodes('nodes.csv')
edges = load_edges('edges.csv')
dynamic_factors = load_dynamic_factors('dynamic_factors.csv')
transfers = load_transfers('transfer.csv')

In [42]:
# Run A* search with dynamic factors
optimal_path = a_star_search_dynamic_print_path(selected_start, selected_goal, nodes, edges, transfers, selected_time_window, dynamic_factors)

print("Optimal Path with Dynamic Factors:", optimal_path)

Considering path from Guwahati to Barpeta with cost 39.45 (mode: highway)
Considering path from Guwahati to Biswanath with cost 52.60000000000001 (mode: highway)
Considering path from Guwahati to North Lakhimpur with cost 69.5 (mode: highway)
Considering path from Guwahati to Sonitpur with cost 75.0 (mode: highway)
Considering path from Guwahati to Golaghat with cost 100.0 (mode: railway)
Path taken: ['Guwahati', 'Barpeta']
Total path cost: 39.45
Optimal Path with Dynamic Factors: ['Guwahati', 'Barpeta']


## UTILITY CODE

In [26]:
#The following is a helper code the make all routes bidirectional
import pandas as pd

# Load the CSV data into a DataFrame
df = pd.read_csv("edges.csv")

# Create a new DataFrame with reversed routes
reversed_df = df.copy()
reversed_df.columns = ['to_node', 'from_node', 'mode', 'cost', 'time']

# Combine original and reversed dataframes
combined_df = pd.concat([df, reversed_df], ignore_index=True)

# Remove duplicate routes (if any)
combined_df = combined_df.drop_duplicates()

# Save the modified DataFrame to a new CSV file
combined_df.to_csv("edges.csv", index=False)

In [34]:
#This code is to change the city name mapping

import pandas as pd

# Define the new city names (various cities and districts of Assam)
new_city_names = [
    'Guwahati', 'Dispur', 'Tezpur', 'Jorhat', 'Dibrugarh', 'Silchar', 'Tinsukia', 'Nagaon', 'Sivasagar', 
    'North Lakhimpur', 'Dhubri', 'Goalpara', 'Barpeta', 'Bongaigaon', 'Karimganj', 'Hailakandi', 
    'Dima Hasao', 'Golaghat', 'Morigaon', 'Nalbari', 'Baksa', 'Chirang', 'Kokrajhar', 'Udalguri', 
    'Sonitpur', 'Biswanath', 'Majuli', 'Hojai', 'Charaideo', 'South Salmara-Mankachar', 'Dhemaji', 
    'Lakhimpur', 'Kamrup', 'Kamrup Metropolitan'
]

# Ensure the number of new city names matches the number of old city names
assert len(new_city_names) == 34, "The number of new city names must match the number of old city names."

# Read the existing nodes data
nodes = pd.read_csv('nodes.csv')

# Create a mapping from old city names to new city names
old_city_names = nodes['node_name'].tolist()
city_name_mapping = dict(zip(old_city_names, new_city_names))

# Function to replace old city names with new city names in a DataFrame
def replace_city_names(df, columns):
    for col in columns:
        df[col] = df[col].map(city_name_mapping)
    return df

# Update nodes.csv
nodes['node_name'] = nodes['node_name'].map(city_name_mapping)
nodes.to_csv('nodes_updated.csv', index=False)

# Update transfer.csv
transfer = pd.read_csv('transfer.csv')
transfer = replace_city_names(transfer, ['node_name'])
transfer.to_csv('transfer_updated.csv', index=False)

# Update dynamic_factors.csv
dynamic_factors = pd.read_csv('dynamic_factors.csv')
dynamic_factors = replace_city_names(dynamic_factors, ['from_node', 'to_node'])
dynamic_factors.to_csv('dynamic_factors_updated.csv', index=False)

# Update edges.csv
edges = pd.read_csv('edges.csv')
edges = replace_city_names(edges, ['from_node', 'to_node'])
edges.to_csv('edges_updated.csv', index=False)

print("City names updated successfully in all files.")

City names updated successfully in all files.


## ANIMATION

In [43]:
import math
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.animation import PillowWriter
import os

In [48]:
#!pip install Pillow


[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [47]:
import matplotlib.pyplot as plt
import networkx as nx
import os
import heapq

class Node:
    def __init__(self, name, g_cost, h_cost, parent=None):
        self.name = name
        self.g_cost = g_cost  # cost from start node
        self.h_cost = h_cost  # heuristic cost to goal node
        self.parent = parent

    def __lt__(self, other):
        return (self.g_cost + self.h_cost) < (other.g_cost + other.h_cost)

class AStarVisualizer:
    def __init__(self, nodes, edges):
        self.G = nx.Graph()
        for node, coord in nodes.items():
            self.G.add_node(node, pos=(coord['y'], coord['x']))
        for node, neighbors in edges.items():
            for neighbor, base_cost, mode, travel_time in neighbors:
                self.G.add_edge(node, neighbor)
        
        self.pos = nx.get_node_attributes(self.G, 'pos')
        self.step = 0
        
        # Create output directory if it doesn't exist
        self.output_dir = "astar_steps"
        os.makedirs(self.output_dir, exist_ok=True)

    def visualize_step(self, current_path, considered_edge, current_node=None, current_cost=None, travel_time=None, mode=None):
        self.step += 1
        fig, ax = plt.subplots(figsize=(12, 8))
        
        # Draw all nodes and edges
        nx.draw(self.G, self.pos, with_labels=True, node_size=500, node_color='skyblue', 
                font_size=10, font_weight='bold', ax=ax)
        
        # Draw the current path in green
        if current_path:
            path_edges = list(zip(current_path[:-1], current_path[1:]))
            nx.draw_networkx_edges(self.G, self.pos, edgelist=path_edges, edge_color='green', width=2, ax=ax)
        
        # Draw the considered edge in red
        if considered_edge:
            nx.draw_networkx_edges(self.G, self.pos, edgelist=[considered_edge], edge_color='red', width=2, ax=ax)
        
        # Add text annotations at the bottom right
        if current_node is not None:
            ax.text(0.95, 0.05, f"Current node: {current_node}", transform=ax.transAxes, fontsize=12, verticalalignment='bottom', horizontalalignment='right')
        
        if current_cost is not None and travel_time is not None and mode is not None:
            ax.text(0.95, 0.10, f"Cost: {current_cost}, Time: {travel_time}, Mode: {mode}", transform=ax.transAxes, fontsize=12, verticalalignment='bottom', horizontalalignment='right')
        
        if current_path:
            ax.text(0.95, 0.15, f"Current path: {' -> '.join(current_path)}", transform=ax.transAxes, fontsize=12, verticalalignment='bottom', horizontalalignment='right')
        
        ax.set_title(f"A* Search - Step {self.step}")
        
        # Save the figure
        plt.savefig(os.path.join(self.output_dir, f"step_{self.step:03d}.png"))
        plt.close(fig)

def a_star_search_dynamic_visualize(start, goal, nodes, edges, transfers, time_window, current_conditions):
    open_list = []
    closed_list = set()
    
    try:
        start_node = Node(start, 0, manhattan_distance(nodes[start], nodes[goal]))
    except KeyError as e:
        print(f"Error: {e} not found in nodes dictionary")
        return None
    
    heapq.heappush(open_list, start_node)
    
    visualizer = AStarVisualizer(nodes, edges)
    visualizer.visualize_step([start], None)  # Initial state
    
    while open_list:
        current_node = heapq.heappop(open_list)
        
        if current_node.name == goal:
            path = []
            total_cost = current_node.g_cost
            while current_node:
                path.append(current_node.name)
                current_node = current_node.parent
            path = path[::-1]
            print(f"Path taken: {path}")
            print(f"Total path cost: {total_cost}")
            
            visualizer.visualize_step(path, None)  # Final path
            return path
        
        closed_list.add(current_node.name)
        
        if current_node.name not in edges:
            continue
        
        current_path = []
        node = current_node
        while node:
            current_path.append(node.name)
            node = node.parent
        current_path = current_path[::-1]
        
        for neighbor, base_cost, mode, travel_time in edges[current_node.name]:
            if neighbor in closed_list:
                continue
            
            adjusted_cost = adjust_costs_dynamic(current_node.name, neighbor, base_cost, current_conditions)
            g_cost = current_node.g_cost + adjusted_cost
            
            if g_cost > time_window:
                continue
            
            h_cost = manhattan_distance(nodes[neighbor], nodes[goal])
            neighbor_node = Node(neighbor, g_cost, h_cost, current_node)
            
            if neighbor_node not in open_list:
                heapq.heappush(open_list, neighbor_node)
                print(f"Considering path from {current_node.name} to {neighbor} with cost {adjusted_cost} (mode: {mode})")
                visualizer.visualize_step(current_path, (current_node.name, neighbor), current_node.name, g_cost, travel_time, mode)
    
    print("No path found")
    return None

In [48]:
# Run A* search with dynamic factors
optimal_path = a_star_search_dynamic_visualize(selected_start, selected_goal, nodes, edges, transfers, selected_time_window, dynamic_factors)

print("Optimal Path with Dynamic Factors:", optimal_path)

Considering path from Guwahati to Barpeta with cost 39.45 (mode: highway)
Considering path from Guwahati to Biswanath with cost 52.60000000000001 (mode: highway)
Considering path from Guwahati to North Lakhimpur with cost 69.5 (mode: highway)
Considering path from Guwahati to Sonitpur with cost 75.0 (mode: highway)
Considering path from Guwahati to Golaghat with cost 100.0 (mode: railway)
Path taken: ['Guwahati', 'Barpeta']
Total path cost: 39.45
Optimal Path with Dynamic Factors: ['Guwahati', 'Barpeta']
